## Fine tune a llama3 model with m-a-p/Code-Feedback

### Setup
Let's start by installing all the lib we need to do supervised fine-tuning. We're going to use

Transformers for the LLM which we're going to fine-tune
Datasets for loading a SFT dataset from the hub, and preparing it for the model
BitsandBytes and PEFT for fine-tuning the model on consumer hardware, leveraging Q-LoRa, a technique which drastically reduces the compute requirements for fine-tuning
TRL, a library which includes useful Trainer classes for LLM fine-tuning.

In [ ]:
#!pip install -q transformers[torch] datasets
#!pip install -q bitsandbytes trl peft
#!pip install flash-attn --no-build-isolation

In [ ]:
# torch.cuda.empty_cache()

### Load Data + Preprocessing
The dataset contains various splits, each with a certain number of rows. In our case, as we're going to do supervised fine-tuning (SFT), only the "train_sft" and "test_sft" splits are relevant for us.


In [ ]:
from datasets import load_dataset, DatasetDict

In [ ]:
raw_datasets = load_dataset("m-a-p/Code-Feedback")
raw_datasets

In [ ]:
# SPLIT data in train/test
indices_1 = range(0,100)
indices_2 = range(101,201)
dataset_dict = {
    "train": raw_datasets["train"].select(indices_1),
    "test": raw_datasets["train"].select(indices_2)
}
raw_dataset = DatasetDict(dataset_dict)
raw_dataset

In [ ]:
raw_dataset["train"][0]

#### Tokenizer
Next, we instantiate the tokenizer, which is required to prepare the text for the model. The model doesn't directly take strings as input, but rather input_ids, which represent integer indices in the vocabulary of a Transformer model. 

We also set some attributes which the tokenizer of a base model typically doesn't have set, such as:

- the padding token ID. During pre-training, one doesn't need to pad since one just creates blocks of text to predict the next token, but during fine-tuning, we will need to pad the (instruction, completion) pairs in order to create batches of equal length.
- the model max length: this is required in order to truncate sequences which are too long for the model. Here we decide to train on at most 2048 tokens.
- the chat template. A chat template determines how each list of messages is turned into a tokenizable string, by adding special strings in between such as <|user|> to indicate a user message and <|assistant|> to indicate the chatbot's response. Here we define the default chat template, used by most chat models. See also the docs.

In [ ]:
from transformers import AutoTokenizer
from huggingface_hub import login

In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

# set pad_token_id equal to the eos_token_id if not set
if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id

# Set reasonable default for models without max length
if tokenizer.model_max_length > 100_000:
  tokenizer.model_max_length = 2048

In [ ]:
DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"
tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE

#### Apply chat template
Once we have equipped the tokenizer with the appropriate attributes, it's time to apply the chat template to each list of messages. Here we basically turn each list of (instruction, completion) messages into a tokenizable string for the model.

Note that we specify tokenize=False here, since the SFTTrainer which we'll define later on will perform the tokenization internally. Here we only turn the list of messages into strings with the same format.

In [ ]:
import re
import random
from multiprocessing import cpu_count


def apply_chat_template(example, tokenizer):
    messages = example["messages"]
    # We add an empty system message if there is none
    if messages[0]["role"] != "system":
        messages.insert(0, {"role": "system", "content": ""})
    example["text"] = tokenizer.apply_chat_template(messages, tokenize=False)

    return example

In [ ]:
column_names = list(raw_dataset["train"].features)
column_names


In [ ]:
# applies the apply_chat_template function to each element in raw_dataset using multiple CPU cores, passing a tokenizer and removing specified columns, with a progress description.
raw_dataset = raw_dataset.map(apply_chat_template,
                                num_proc=cpu_count(),
                                fn_kwargs={"tokenizer": tokenizer},
                                remove_columns=column_names,
                                desc="Applying chat template")

In [ ]:
train_data = raw_dataset["train"]
test_data = raw_dataset["test"]

for index in random.sample(range(len(raw_dataset["train"])), 3):
  print(f"Sample {index} of the processed training set:\n\n{raw_dataset['train'][index]['text']}")
  

### Model Definition

With regular LoRa, one would keep the base model in 32 or 16 bits in memory, and then train the parameter weights. However, there have been new methods developed to shrink the size of a model considerably, to 8 or 4 bits per parameter (we call this "quantization"). Hence, if we apply LoRa to a quantized model (like a 4-bit model), then we call this QLoRa. We have a blog post that tells you all about it. There are various quantization methods available, here we're going to use the BitsandBytes integration.



In [ ]:
# QloRa SFT
from transformers import BitsAndBytesConfig
import torch

quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
)

device_map = {"": torch.cuda.current_device()} if torch.cuda.is_available() else None
device_map


In [ ]:
model_kwargs = dict(
    #attnt_implementation=True,
    torch_dtype="auto",
    use_cache=False, # set to False as we're going to use gradient checkpointing
    device_map=device_map,
    quantization_config=quant_config.to_dict() 
)



### SFT Trainer

Next, we define the SFTTrainer available in the TRL library. This class inherits from the Trainer class available in the Transformers library, but is specifically optimized for supervised fine-tuning (instruction tuning). It can be used to train out-of-the-box on one or more GPUs, using Accelerate as backend.

Most notably, it supports packing, where multiple short examples are packed in the same input sequence to increase training efficiency.

As we're going to use QLoRa, the PEFT library provides a handy LoraConfig which defines on which layers of the base model to apply the adapters. One typically applies LoRa on the linear projection matrices of the attention layers of a Transformer. We then provide this configuration to the SFTTrainer class. The weights of the base model will be loaded as we specify the model_id (this requires some time).

We also specify various hyperparameters regarding training, such as:

- we're going to fine-tune for 1 epoch
- the learning rate and its scheduler
- we're going to use gradient checkpointing (yet another way to save memory during training)
- and so on.

In [ ]:
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig
from transformers import TrainingArguments

In [ ]:
output_dir = "../../model_saved/llama_code_feedback-8B"

In [ ]:
# based on config
training_args = TrainingArguments(
    fp16=True, # specify bf16=True instead when training on GPUs that support bf16
    do_eval=True,
    eval_strategy="epoch",
    gradient_accumulation_steps=128,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    learning_rate=2.0e-05,
    log_level="info",
    logging_steps=5,
    logging_strategy="steps",
    lr_scheduler_type="cosine",
    max_steps=-1,
    num_train_epochs=1,
    output_dir=output_dir,
    overwrite_output_dir=True,
    per_device_eval_batch_size=1, # originally set to 8
    per_device_train_batch_size=1, # originally set to 8
    save_strategy="no",
    save_total_limit=None,
    seed=42,
)

# based on config
peft_config = LoraConfig(
        r=64,
        lora_alpha=16,
        lora_dropout=0.1,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

In [ ]:
trainer = SFTTrainer(
        model=model_id,
        model_init_kwargs=model_kwargs,
        args=training_args,
        train_dataset=train_data,
        eval_dataset=test_data,
        dataset_text_field="text",
        tokenizer=tokenizer,
        packing=True,
        peft_config=peft_config,
        max_seq_length=tokenizer.model_max_length,
    )

In [ ]:
train_result = trainer.train()

### Saving Model

In [ ]:
metrics = train_result.metrics
max_train_samples = 100
metrics["train_samples"] = min(max_train_samples, len(train_data))
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()
trainer.save_model(training_args.output_dir)

### Inference

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

output_dir = "../../model_saved/llama_code_feedback-8B"

tokenizer = AutoTokenizer.from_pretrained(output_dir)
model = AutoModelForCausalLM.from_pretrained(output_dir, load_in_4bit=True, device_map="auto")

In [ ]:
import torch

# We use the tokenizer's chat template to format each message
messages = [
    {"role": "user", "content": "Write a Ruby code to convert a double-linked list to a single-linked list"},
]

DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '' }}\n{% endif %}\n{% endfor %}"
tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE

# Prepare the messages for the model
input_ids = tokenizer.apply_chat_template(messages, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")

# Ensure input_ids are valid
print("Input IDs:", input_ids)

# Inference with model output checks
outputs = model.generate(
        input_ids=input_ids,
        max_new_tokens=256,
        do_sample=True,
        temperature=1.0,  # Adjusted for stability
        top_k=50,
        top_p=0.9,
        return_dict_in_generate=True,
        output_scores=True
)

# Decode the generated output
decoded_output = tokenizer.batch_decode(outputs['sequences'], skip_special_tokens=True)[0]
print(decoded_output)
